In [2]:

import requests
from bs4 import BeautifulSoup

GENERIC_XML_URL = "https://cdn.littlefox.co.kr/cn/captionxml/C0001143.xml"


In [21]:
class Series:
    def __init__(self, series_id):
        self.main_url = f'https://chinese.littlefox.com/en/story/contents_list/{series_id}'

    def get_page_count(self):
        response = requests.get(self.main_url)
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        paging_div = soup.find('div', class_='lf_paging')

        if paging_div:
            page_numbers = []
            for a_tag in paging_div.find_all('a'):
                try:
                    page_number = int(a_tag.text)
                    page_numbers.append(page_number)
                except ValueError:
                    # Ignore non-numeric values
                    continue
            if page_numbers:
                max_page_number = max(page_numbers)
                return max_page_number
        return None

    def get_page_urls(self):
        max_page_count = self.get_page_count()
        if max_page_count:
            page_urls = [f'{self.main_url}?&page={page}' for page in range(1, max_page_count + 1)]
            return page_urls
        else:
            print("No pages found")
            return None

    def get_ep_ids(self):
        page_urls = self.get_page_urls()
        print(page_urls)
        ids = []
        for url in page_urls: 
            response = requests.get(url)
            html_content = response.text
            soup = BeautifulSoup(html_content, 'html.parser')
            items = soup.find_all('div', class_='item')
            for item in items:
                input_element = item.find('input', class_='LF_CHK s2 contentsCheck')
                if input_element:
                    value = input_element.get('value')
                    ids.append(value)
        return ids


In [22]:
single_stories_1 = Series('DP000801')
single_stories_1.get_ep_ids()

['C0008512',
 'C0008513',
 'C0008514',
 'C0008515',
 'C0008516',
 'C0008517',
 'C0008518',
 'C0008519',
 'C0008520',
 'C0008521',
 'C0008522',
 'C0008523',
 'C0008524',
 'C0008525',
 'C0008526',
 'C0008527',
 'C0008528',
 'C0008529',
 'C0008530',
 'C0008531',
 'C0008532',
 'C0008533',
 'C0008534',
 'C0008535']